In [1]:
import random
import datetime as dt

from job_client_graphql import Client, CellDisplay

## Prerequisite
+ Launch JOB server - see [documentation](TBD) section **5-Examples**

In [4]:
pages = {
    'hello-world': 'https://observablehq.com/d/35986e2e25619d6e',
}

In [5]:
s = Client(server_url='http://localhost:4001/graphql',
           default_channel='channel-1',
           password='mysecret',
           pages=pages,         
           debug=False)
print(s.build_url('hello-world'))
s.show_btn_page('hello-world')

https://observablehq.com/d/35986e2e25619d6e?host=localhost&port=4001&path=/graphql&ssl=False&channel=channel-1


## write

In [31]:
def rnd_data():
    return ''.join(random.choices('abcdefghijk', k=10)) + str(random.randint(0,1e6))

In [32]:
name, value = 'toto', rnd_data()
print(name, value)
s.write(name, value)

toto edfcdgfihf315552


True

In [33]:
name, value = 'titi', rnd_data()
print(name, value)
s.write(name, value)

titi ieejeejjjc227916


True

In [34]:
name, value = 'tata', rnd_data()
print(name, value)
s.write(name, value, expiry=10) # self delete after 10s

tata bakadeadhe764415


True

## publish

In [37]:
name, value = 'tutu', rnd_data() 
print(name, value)
s.publish(name, value)

tutu fgjagddjag115068


True

## publishWrite

In [40]:
name, value = 'tete', rnd_data()
print(name, value)
s.publish_write(name, value, expiry=30) # self delete after 30s

tete igfbbkdiij10843


True

## read

In [41]:
name = 'toto'
s.read(name)

{'a': 173,
 'b': 165.46045327673835,
 'c': 'zxlctIkqb5',
 'd': {'aa': 169, 'bb': 162.45033459436542, 'cc': 'gTyeHWRQ9C'}}

In [42]:
name = 'titi'
s.read(name)

'D8i44Inrmz'

In [43]:
name = 'tata'
s.read(name)

## readHisto

In [44]:
name = 'toto'
s.read_histo(name, 4, timestamp=True)

[{'value': 163.6029165124119,
  'timestamp': datetime.datetime(2020, 5, 15, 19, 30, 6, 26000)},
 {'value': {'a': 170,
   'b': 197.5911226799439,
   'c': 'pNK6CtiSti',
   'd': {'aa': 173, 'bb': 135.45567707446182, 'cc': 'dzW4lscJx2'}},
  'timestamp': datetime.datetime(2020, 5, 15, 19, 30, 5, 26000)},
 {'value': 161.5016939553714,
  'timestamp': datetime.datetime(2020, 5, 15, 19, 30, 3, 23000)},
 {'value': {'a': 148,
   'b': 198.62843140504827,
   'c': '3VtbpyvlEY',
   'd': {'aa': 101, 'bb': 103.27873059389847, 'cc': 'omBbwZJaUX'}},
  'timestamp': datetime.datetime(2020, 5, 15, 19, 30, 1, 19000)}]

In [45]:
name = 'titi'
s.read_histo(name, 4, timestamp=False)

[174.54825229005854,
 191,
 'D8i44Inrmz',
 {'a': 116,
  'b': 110.31333075406593,
  'c': '1z6t6GRuOB',
  'd': {'aa': 103, 'bb': 126.29993142784897, 'cc': 'BFX8YDPPIu'}}]

## Store

Client.store contains the latest values read

In [46]:
s.store

{'channel-1': {'toto': {'a': 173,
   'b': 165.46045327673835,
   'c': 'zxlctIkqb5',
   'd': {'aa': 169, 'bb': 162.45033459436542, 'cc': 'gTyeHWRQ9C'}},
  'titi': 'D8i44Inrmz',
  'tata': None}}

Client.update contains the sequence of updates (channel, name, value) - starting from latest values read


In [47]:
s.update[:3]

[('channel-1',
  'toto',
  {'a': 173,
   'b': 165.46045327673835,
   'c': 'zxlctIkqb5',
   'd': {'aa': 169, 'bb': 162.45033459436542, 'cc': 'gTyeHWRQ9C'}}),
 ('channel-1', 'titi', 'D8i44Inrmz'),
 ('channel-1', 'tata', None)]

## Subscribe

### DISPLAY
In this demo (channel:name->value) are displayed upon udpate

In [50]:
out = CellDisplay(name='subscribe-status', max_lines=5)
# show_lines = []

def show(channel, name, value):
    ts = dt.datetime.now().strftime('%H:%M:%S.%f')[:-4]
    s = f'received on {channel}: {name} --> {value} ({ts})'
    out.update(s, kind='text', append=True)

received on channel-1: toto --> "mDjQ8GNb9I" (21:30:42.08)
received on channel-1: titi --> 110 (21:30:43.07)
received on channel-1: toto --> {"a":160,"b":149.61390546485103,"c":"vLHbBMHVHr","d":{"aa":113,"bb":170.37354542147676,"cc":"EfCkaRqvKe"}} (21:30:44.07)
received on channel-1: toto --> {"a":182,"b":152.5025211858453,"c":"UtOdXYnATS","d":{"aa":112,"bb":163.00626054799307,"cc":"4nXImwPqzj"}} (21:30:45.07)
received on channel-1: toto --> 145.36042870786775 (21:30:46.07)

In [51]:
out.display()

received on channel-1: toto --> "mDjQ8GNb9I" (21:30:42.08)
received on channel-1: titi --> 110 (21:30:43.07)
received on channel-1: toto --> {"a":160,"b":149.61390546485103,"c":"vLHbBMHVHr","d":{"aa":113,"bb":170.37354542147676,"cc":"EfCkaRqvKe"}} (21:30:44.07)
received on channel-1: toto --> {"a":182,"b":152.5025211858453,"c":"UtOdXYnATS","d":{"aa":112,"bb":163.00626054799307,"cc":"4nXImwPqzj"}} (21:30:45.07)
received on channel-1: toto --> 145.36042870786775 (21:30:46.07)

### START
If no channel is specified the Client.default_channel is used.

In [52]:
ref = s.subscribe(action=show)

### STOP

In [53]:
s.subscribe_stop(ref)

subscription ref=XbBNY8 stopped


In [24]:
out.clear()